In [1]:
import numpy as np
import theano
import theano.tensor as T

from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
from theano.tensor.nnet.conv import conv2d
from theano.tensor.signal.downsample import max_pool_2d

%matplotlib inline

In [2]:
def rolling_window(a, window):
    """
    Make an ndarray with a rolling window of the last dimension

    Parameters
    ----------
    a : array_like
        Array to add rolling window to
    window : int
        Size of rolling window

    Returns
    -------
    Array that is a view of the original array with a added dimension
    of size w.

    Examples
    --------
    >>> x=np.arange(10).reshape((2,5))
    >>> rolling_window(x, 3)
    array([[[0, 1, 2], [1, 2, 3], [2, 3, 4]],
           [[5, 6, 7], [6, 7, 8], [7, 8, 9]]])

    Calculate rolling mean of last dimension:

    >>> np.mean(rolling_window(x, 3), -1)
    array([[ 1.,  2.,  3.],
           [ 6.,  7.,  8.]])

    """
    assert window >= 1, "`window` must be at least 1."
    assert window < a.shape[-1], "`window` is too long."

    # # with strides
    shape = a.shape[:-1] + (a.shape[-1] - window, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

In [3]:
def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

def init_weights(shape):
    return theano.shared(floatX(np.random.randn(*shape) * 0.01))

def rectify(X):
    return T.maximum(X, 0.)

def softmax(X):
    e_x = T.exp(X - X.max(axis=1).dimshuffle(0, 'x'))
    return e_x / e_x.sum(axis=1).dimshuffle(0, 'x')

def softrect(X):
    return T.log(1 + T.exp(X))

def poiss(rhat, r, fudge=1e-8):
    return rhat - r * T.log(rhat+fudge)

def dropout(X, p=0.):
    if p > 0:
        retain_prob = 1 - p
        X *= srng.binomial(X.shape, p=retain_prob, dtype=theano.config.floatX)
        X /= retain_prob
    return X

def RMSprop(cost, params, lr=0.001, rho=0.9, epsilon=1e-6):
    grads = T.grad(cost=cost, wrt=params)
    updates = []
    for p, g in zip(params, grads):
        acc = theano.shared(p.get_value() * 0.)
        acc_new = rho * acc + (1 - rho) * g ** 2
        gradient_scaling = T.sqrt(acc_new + epsilon)
        g = g / gradient_scaling
        updates.append((acc, acc_new))
        updates.append((p, p - lr * g))
    return updates

def model(X, w1, w2):
    l1 = rectify(conv2d(X, w1, border_mode='full'))
    y = softrect(T.dot(T.flatten(l1, outdim=2),w2))
    return y

# simulate a toy conv-LN model

In [4]:
# fake filter
filter_dims = (5,5,10)
xm, ym = np.meshgrid(np.linspace(-5,5,filter_dims[0]), np.linspace(-5,5,filter_dims[1]))

z = xm**2 + ym**2
z = np.abs(z.max() - z)

f_star = np.outer(z, np.sin(np.linspace(0, 2*np.pi, filter_dims[2]))).reshape(5,5,-1)

In [5]:
# generate stimulus
num_samples = 100
stim_size = (32,32)

stim = np.random.randn(*(stim_size + (num_samples,)))

In [6]:
w1 = init_weights((1,) + f_star.T.shape)
w2 = init_weights((np.prod(stim_size),1))

X = T.ftensor4()
r = T.fvector()

rhat = model(X, w1, w2)
cost = T.mean(poiss(r,rhat))

updates = RMSprop(cost, [w1,w2], lr=0.01)

train = theano.function(inputs=[X,r], outputs=cost, updates=updates, allow_input_downcast=True)

In [7]:
w_star = floatX(np.zeros((np.prod(stim_size),1)))
w_star[170,0] = 1
pred = theano.function(inputs=[X], outputs=model(X,floatX(f_star.T.reshape(1,10,5,5,)),w_star), allow_input_downcast=True)

In [8]:
stim = np.random.randn(1, 10, 32, 32)
rhat = pred(stim)

ValueError: Shape mismatch: A.shape[1] != y.shape[0]
Apply node that caused the error: CGemv{inplace}(Alloc.0, TensorConstant{1.0}, Flatten{2}.0, TensorConstant{[ 0.  0.  ...  0.  0.]}, TensorConstant{0.0})
Inputs shapes: [(1,), (), (1, 1296), (1024,), ()]
Inputs strides: [(4,), (), (5184, 4), (4,), ()]
Inputs types: [TensorType(float32, vector), TensorType(float32, scalar), TensorType(float32, matrix), TensorType(float32, vector), TensorType(float32, scalar)]
Use the Theano flag 'exception_verbosity=high' for a debugprint of this apply node.

In [ ]:
cost_history = list()
for j in range(10000):

    # generate spiking data
    stim = np.rollaxis(np.rollaxis(rolling_window(stim, filter_dims[2]),3),3)
    rhat = pred(stim)

# linear regression example

In [106]:
# toy example
w = init_weights(f_star.shape)

(100,)

In [111]:
x = T.ftensor4()
y = T.fvector()

In [125]:
def sqerr(x, y):
    u = T.tensordot(w, x, ([0,1,2],[0,1,2]))
    return 0.5*((y-u)**2)

cost = T.mean(sqerr(x,y))
updates = RMSprop(cost, [w], lr=0.1)

In [126]:
train = theano.function(inputs=[x, y], outputs=cost, updates=updates, allow_input_downcast=True)

In [128]:
cost_history = list()
for j in range(2500):
    
    num_samples = 1000
    
    # generate data
    X = np.random.randn(*(f_star.shape + (num_samples,)))
    Y = np.tensordot(f_star, X, ([0,1,2], [0,1,2])) + 0.01*np.random.randn(num_samples)
    
    cost_history.append(train(X,Y))
    
    if j % 25 == 0:
        print('[%i] Cost = %f' % (j, cost_history[-1]))

[0] Cost = 19389.595703
[25] Cost = 15695.724609
[50] Cost = 13127.696289
[75] Cost = 10532.672852
[100] Cost = 7178.458496
[125] Cost = 5844.898438
[150] Cost = 3739.792236
[175] Cost = 2879.681152
[200] Cost = 1881.470825
[225] Cost = 1247.353638
[250] Cost = 771.851990
[275] Cost = 413.708069
[300] Cost = 201.289230
[325] Cost = 92.907364
[350] Cost = 36.531059
[375] Cost = 12.850611
[400] Cost = 2.750138
[425] Cost = 0.269782
[450] Cost = 0.359387
[475] Cost = 0.327884
[500] Cost = 0.448778
[525] Cost = 0.425444
[550] Cost = 0.305991
[575] Cost = 0.329312
[600] Cost = 0.379528
[625] Cost = 0.365201
[650] Cost = 0.264057
[675] Cost = 0.314834
[700] Cost = 0.424314
[725] Cost = 0.397663
[750] Cost = 0.265900
[775] Cost = 0.312521
[800] Cost = 0.337047
[825] Cost = 0.342705
[850] Cost = 0.340496
[875] Cost = 0.289348
[900] Cost = 0.237004
[925] Cost = 0.299945
[950] Cost = 0.304594
[975] Cost = 0.415855
[1000] Cost = 0.447401
[1025] Cost = 0.303455
[1050] Cost = 0.531838
[1075] Cost =